In [1]:
import datetime
from icalendar import Calendar
import pandas as pd

In [6]:
def ics_to_dataframe(file_path):
    # Open and parse the .ics file
    with open(file_path, 'rb') as f:
        cal = Calendar.from_ical(f.read())

    # Prepare a list to hold event data
    events = []

    for component in cal.walk():
        if component.name == "VEVENT":
            # Extract relevant data from each event
            try:
                title, authors = component.get('summary').split(" - ", 1)  # Expecting exactly 2 items
                authors = authors.split('\,')
            except ValueError:  # If not exactly 2 items, then it's a ValueError
                title = component.get('summary')  # Take the whole summary as title
                authors = None  # Default, no authors found
                
            start_dt = component.get('dtstart').dt
            end_dt = component.get('dtend').dt
            title = component.get('summary')
            description = component.get('description')

            if isinstance(start_dt, datetime.datetime) and isinstance(end_dt, datetime.datetime):
                # Calculate duration
                duration = (end_dt - start_dt).total_seconds() / 60  # duration in minutes
                # Formatting date and start time
                date = start_dt.strftime('%Y-%m-%d')
                start_time = start_dt.strftime('%H:%M')
                

                # Append event info to the list
                events.append([date, start_time, duration, title, authors])

    # Create a DataFrame
    df = pd.DataFrame(events, columns=['date', 'start time', 'duration_mins', 'title', 'authors'])

    return df

file_path = "data/event-calendar-2022.ics"
df = ics_to_dataframe(file_path)
df.head()

,date,start time,duration_mins,title,authors
0,2022-05-18,02:00,4.0,[MSR Technical Papers] An Empirical Evaluation...,"[Nhan Nguyen, Sarah Nadi]"
1,2022-05-18,02:04,4.0,[MSR Technical Papers] Comments on Comments: W...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince..."
2,2022-05-18,02:08,7.0,[MSR Technical Papers] Does This Apply to Me? ...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ..."
3,2022-05-18,02:15,7.0,[MSR Technical Papers] Towards Reliable Agile ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan..."
4,2022-05-18,02:22,7.0,[MSR Technical Papers] BotHunter: An Approach ...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein..."


In [10]:
def parse_sessions(file_path):
    with open(file_path, 'rb') as f:
        cal = Calendar.from_ical(f.read())

    sessions = {}
    
    for component in cal.walk('vevent'):
        start_dt = component.get('dtstart').dt
        end_dt = component.get('dtend').dt
        summary = str(component.get('summary'))
        session_name = summary.split(': ')[-1]  # Assuming session name is after last colon
        
        # Key by start date, end date, and location for simplicity; tweak as needed
        key = (start_dt, end_dt)
        sessions[key] = session_name
    
    return sessions

# Function to find a session for a given event
def find_session_for_event(sessions, event_start, event_end):
    # Note: This simplistic approach assumes exact match of start/end times. 
    # Consider more complex logic if sessions/events don't align exactly.
    key = (event_start, event_end)
    return sessions.get(key, "Unknown Session")

# Load sessions
session_calendar_path = "data/session-calendar-2022.ics"
sessions = parse_sessions(session_calendar_path)

# Map events to sessions and add a new column
df['session_name'] = df.apply(lambda row: find_session_for_event(sessions, 
                                             datetime.datetime.strptime(row['date'] + ' ' + row['start time'], '%Y-%m-%d %H:%M'),
                                             datetime.datetime.strptime(row['date'] + ' ' + row['start time'], '%Y-%m-%d %H:%M') + datetime.timedelta(minutes=row['duration_mins'])),
                                             axis=1)

df.head()

,date,start time,duration_mins,title,authors,session_name
0,2022-05-18,02:00,4.0,[MSR Technical Papers] An Empirical Evaluation...,"[Nhan Nguyen, Sarah Nadi]",Unknown Session
1,2022-05-18,02:04,4.0,[MSR Technical Papers] Comments on Comments: W...,"[Nikitha Rao, Jason Tsay, Martin Hirzel, Vince...",Unknown Session
2,2022-05-18,02:08,7.0,[MSR Technical Papers] Does This Apply to Me? ...,"[Akalanka Galappaththi, Sarah Nadi, Christoph ...",Unknown Session
3,2022-05-18,02:15,7.0,[MSR Technical Papers] Towards Reliable Agile ...,"[Jirat Pasuksmit, Patanamon Thongtanunam, Shan...",Unknown Session
4,2022-05-18,02:22,7.0,[MSR Technical Papers] BotHunter: An Approach ...,"[Ahmad Abdellatif, Mairieli Wessel, Igor Stein...",Unknown Session
